# McStasScript
McStasScript allows writing and running McStas simulations directly from python. The API covers almost all features of McStas 2.X, but lacks a few for McStas 3.X releases.

Written by Mads Bertelsen while working at ESS DMSC

Funded by PaNOSC and a part of PaNOSC work package 5.

### Installation
McStasScript is open source and available here: https://github.com/PaNOSC-ViNYL/McStasScript

Easy to install, install McStas as normal and then McStasScript can be installed with:
```
pip install McStasScript --upgrade
```

![normal](figures/Slide1.png)

![McStasScript](figures/Slide2.png)

# Live demo of McStasScript
Here we will set up an instrument simulation together and execute it from the slides!
First we import the package.

In [ ]:
from mcstasscript.interface import instr, functions, plotter

## Configuration
Before using McStasScript for the first time, it needs to be configured to find your McStas and/or McXtrace installation.

In [ ]:
configurator = functions.Configurator()
configurator.set_mcrun_path("/Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1/bin/")
configurator.set_mcstas_path("/Applications/McStas-2.7.1.app/Contents/Resources/mcstas/2.7.1")

## Create instrument objects
The main class is the McStas_instr, used for creating instrument objects.

In [ ]:
instrument = instr.McStas_instr("demo", author="Mads Bertelsen")

## Help from the instrument object
The instrument object knows about your available McStas components, and has help methods available.

In [ ]:
instrument.show_components()

In [ ]:
instrument.show_components("sources")

In [ ]:
instrument.component_help("Moderator")

## Adding components
McStas simulations rely on a sequence of components, these can be added to the instrument object with the *add_component* method. These return an instrument object that can be used to modify the component further.

In [ ]:
src = instrument.add_component("source", "Source_div")
instrument.print_components()

In [ ]:
print(src)

The component parameters become python attributes on the component object.

In [ ]:
src.xwidth = 0.1
src.yheight = 0.05
src.focus_aw = 1.2
src.focus_ah = 2.3
print(src)

Notice the attributes on component objects auto complete and wrong attibutes are not allowed.

In [ ]:
src.focus_ah = 6

### All information about a component
The *show_parameters* method is good for getting an overview of the current state of the component parameters, as it will also show the default values.

In [ ]:
src.show_parameters()

## Add instrument parameters
Its also possible to add instrument parameters, and these can be used directly when setting component parameters.

In [ ]:
instrument.add_parameter("wavelength", value=2.0, comment="Wavelength in [Ang]")
instrument.add_parameter("int", "order", value=1, comment="Monochromator order, integer")
instrument.show_parameters()

In [ ]:
src.lambda0 = "wavelength"
src.dlambda = "0.01*wavelength"
print(src)

### Setting component positions
When creating a new component, one can specify its position and rotation. 

In [ ]:
guide = instrument.add_component("guide", "Guide_gravity", AT=[0,0,1], RELATIVE=src)

In [ ]:
guide.w1 = 0.05
guide.h1 = 0.05
guide.l = 8.0
guide.m = 3.5
guide.G = -9.82

print(guide)

One can always set the positon and rotation of component objects with the appropriate methods at a later point.

In [ ]:
guide.set_AT([0, 0, 2], RELATIVE="source")

### Adding initialize code
Its possible to add lines of C code to the initialize section of the written McStas file, but its equally possible to just perform these calculations directly in python. Here I show how calculate rotation of a monochromator before we add such a component.

In [ ]:
instrument.add_declare_var("double", "mono_Q", value=2*3.14159/1.714) # Q for Ge 311
instrument.add_declare_var("double", "wavevector")
instrument.append_initialize("wavevector = 2.0*PI/wavelength;")

instrument.add_declare_var("double", "mono_rotation")
instrument.append_initialize("mono_rotation = asin(mono_Q/(2.0*wavevector))*RAD2DEG;")
instrument.append_initialize('printf("monochromator rotation = %g deg\\n", mono_rotation);')

### Adding a monochromator
We can now add a monochromator that use the calculated rotation.

In [ ]:
mono = instrument.add_component("mono", "Monochromator_flat")

In [ ]:
mono.zwidth = 0.05
mono.yheight = 0.08
mono.Q = "mono_Q"
mono.set_AT([0, 0, 8.5], RELATIVE=guide)
mono.set_ROTATED([0, "mono_rotation", 0], RELATIVE="guide")

print(mono)

Need to set the beam direction to the direction of the scattered beam

In [ ]:
beam_direction = instrument.add_component("beam_dir", "Arm", AT_RELATIVE=mono)
beam_direction.set_ROTATED([0, "mono_rotation", 0], RELATIVE=mono)

### Adding a sample
Now we can set up a simple powder sample to get some mildly interesting data.

In [ ]:
sample = instrument.add_component("sample", "PowderN", AT=[0,0,1.1], RELATIVE="beam_dir")

In [ ]:
sample.radius = 0.015
sample.yheight = 0.05
sample.reflections = '"Na2Ca3Al2F14.laz"'
print(sample)

### Adding a few monitors
At the end we add a few monitors, a two theta banana detector and a transmission PSD.

In [ ]:
banana = instrument.add_component("banana", "Monitor_nD", RELATIVE=sample)
banana.xwidth = 2.0
banana.yheight = 0.3
banana.restore_neutron = 1
banana.filename = '"banana.dat"'
banana.options = '"theta limits=[5 175] bins=150, banana"'

mon = instrument.add_component("monitor", "PSD_monitor")
mon.nx = 100
mon.ny = 100
mon.filename = '"psd.dat"'
mon.xwidth = 0.05
mon.yheight = 0.08
mon.restore_neutron = 1
mon.set_AT([0,0,0.1], RELATIVE=sample)

### Checking our instrument object
Before running our simulation we can check the contents of the instrument object and ensure its reasonable.

In [ ]:
instrument.print_components()

In [ ]:
instrument.show_parameters()

### Runnig our simulation
Now we can run the simulation directly from python.

In [ ]:
data = instrument.run_full_instrument(ncount=5E6, foldername="data_folder/mcstas_basics",
                                      increment_folder_name=True,
                                      parameters={"wavelength" : 2.8})

### Plotting the data
The *run_full_instrument* method returns a data object that we can plot.

In [ ]:
print(data)

In [ ]:
plotter.make_sub_plot(data, fontsize=16, figsize=(25, 8))

### Setting plotting options
Its also possible to adjust the way data is plotted, and these preferences are stored in the data object.

In [ ]:
functions.name_plot_options("banana", data, left_lim=90, right_lim=150)
functions.name_plot_options("monitor", data, log=True, orders_of_mag=3)
plotter.make_sub_plot(data, fontsize=16)

### Access to the underlying data
The underlying data is read from the McStas data files and imported as numpy arrays.

In [ ]:
import numpy as np
banana = functions.name_search("banana", data)
print(banana)

In [ ]:
print(banana.Intensity[1:10])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(banana.xaxis, banana.Intensity)
plt.xlabel("Two theta [deg]")
plt.ylabel("Intensity [n/s/bin]")

### Instrument visualization
With local installations its possible to run visualization of the instrument.

In [ ]:
instrument.show_instrument(parameters={"wavelength": 2.0})

## Final remarks
McStasScript provides a new way to use McStas from python!

A full tutorial exist as Jupyter Notebooks, this presentation was a condensed version of the first part called McStas_basics.
The tutorial covers both how to use McStasScript and the Union components.

You can get access to the tutorial as part of our IKON Python course, or by cloning the McStasScript github repo.

There is a major release of McStasScript around the corner that will change the syntax slightly, mainly around running the simulation. The documentation was reworked for the upcoming release [https://mads-bertelsen.github.io](https://mads-bertelsen.github.io), old documentation available on github as pdf [https://github.com/PaNOSC-ViNYL/McStasScript](https://github.com/PaNOSC-ViNYL/McStasScript).


## Exercise

Translate a simple instrument file to McStasScript manually.

Use a for-loop in python to add a small series of similar components, for example guide pieces.